<a href="https://colab.research.google.com/github/ledai091/ner/blob/main/version1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/ledai091/ner.git
import os
os.chdir("./ner")

Cloning into 'ner'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 0), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (7/7), 3.06 MiB | 3.67 MiB/s, done.


In [11]:
import pandas as pd
data = pd.read_csv("ner_dataset.csv", encoding="unicode_escape")
data

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [12]:
data['POS'].unique()

array(['NNS', 'IN', 'VBP', 'VBN', 'NNP', 'TO', 'VB', 'DT', 'NN', 'CC',
       'JJ', '.', 'VBD', 'WP', '``', 'CD', 'PRP', 'VBZ', 'POS', 'VBG',
       'RB', ',', 'WRB', 'PRP$', 'MD', 'WDT', 'JJR', ':', 'JJS', 'WP$',
       'RP', 'PDT', 'NNPS', 'EX', 'RBS', 'LRB', 'RRB', '$', 'RBR', ';',
       'UH', 'FW'], dtype=object)

In [13]:
from itertools import chain
def get_dict_map(data, token_or_tag):
    tok2idx = {}
    idx2tok = {}

    if token_or_tag == 'token':
        vocab = list(set(data['Word'].to_list()))
    else:
        vocab = list(set(data['Tag'].to_list()))

    idx2tok = {idx:tok for idx, tok  in enumerate(vocab)}
    tok2idx = {tok:idx for idx, tok  in enumerate(vocab)}
    return tok2idx, idx2tok

token2idx, idx2token = get_dict_map(data, 'token')
tag2idx, idx2tag = get_dict_map(data, 'tag')

In [14]:
tag2idx

{'I-nat': 0,
 'I-per': 1,
 'I-org': 2,
 'B-geo': 3,
 'I-geo': 4,
 'O': 5,
 'B-org': 6,
 'B-tim': 7,
 'B-art': 8,
 'B-gpe': 9,
 'I-gpe': 10,
 'B-eve': 11,
 'I-eve': 12,
 'I-art': 13,
 'B-per': 14,
 'B-nat': 15,
 'I-tim': 16}

In [15]:
# Ánh xạ từng 'Word' trong DataFrame 'data' sang chỉ số tương ứng trong từ điển 'token2idx'.
data['Word_idx'] = data['Word'].map(token2idx)

# Ánh xạ từng 'Tag' trong DataFrame 'data' sang chỉ số tương ứng trong từ điển 'tag2idx'.
data['Tag_idx'] = data['Tag'].map(tag2idx)

# Điền các giá trị NaN trong DataFrame 'data' bằng giá trị hợp lệ liền kề trước đó.
# Điều này giữ cho thông tin của một câu liên tục không bị gián đoạn bởi các giá trị thiếu.
data_fillna = data.fillna(method='ffill', axis=0)

# Nhóm dữ liệu theo 'Sentence #' để có thể xử lý từng câu một cách độc lập.
# 'as_index = False' giữ cho DataFrame sau khi nhóm không sử dụng 'Sentence #' làm chỉ số.
# Với mỗi nhóm, chuyển các cột 'Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx' thành danh sách.
# Điều này tạo ra một cấu trúc dữ liệu mà mỗi câu là một mảng của từ, tags, và chỉ số tương ứng.
data_group = data_fillna.groupby(
    ['Sentence #'], as_index=False
)['Word', 'POS', 'Tag', 'Word_idx', 'Tag_idx'].agg(lambda x: list(x))


<ipython-input-15-119f9c6c33b2>:15: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  data_group = data_fillna.groupby(


In [16]:
data_group

,Sentence #,Word,POS,Tag,Word_idx,Tag_idx
0,Sentence: 1,"[Thousands, of, demonstrators, have, marched, ...","[NNS, IN, NNS, VBP, VBN, IN, NNP, TO, VB, DT, ...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[26367, 155, 16268, 1271, 17425, 2547, 28817, ...","[5, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 3, 5, 5, ..."
1,Sentence: 10,"[Iranian, officials, say, they, expect, to, ge...","[JJ, NNS, VBP, PRP, VBP, TO, VB, NN, TO, JJ, J...","[B-gpe, O, O, O, O, O, O, O, O, O, O, O, O, O,...","[15322, 19966, 14946, 27482, 3944, 19694, 1425...","[9, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."
2,Sentence: 100,"[Helicopter, gunships, Saturday, pounded, mili...","[NN, NNS, NNP, VBD, JJ, NNS, IN, DT, NNP, JJ, ...","[O, O, B-tim, O, O, O, O, O, B-geo, O, O, O, O...","[15284, 13485, 25212, 10938, 12199, 17146, 312...","[5, 5, 7, 5, 5, 5, 5, 5, 3, 5, 5, 5, 5, 5, 6, ..."
3,Sentence: 1000,"[They, left, after, a, tense, hour-long, stand...","[PRP, VBD, IN, DT, NN, JJ, NN, IN, NN, NNS, .]","[O, O, O, O, O, O, O, O, O, O, O]","[6603, 25510, 10507, 21624, 28064, 3492, 3166,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]"
4,Sentence: 10000,"[U.N., relief, coordinator, Jan, Egeland, said...","[NNP, NN, NN, NNP, NNP, VBD, NNP, ,, NNP, ,, J...","[B-geo, O, O, B-per, I-per, O, B-tim, O, B-geo...","[634, 12482, 27782, 16056, 3805, 25609, 25268,...","[3, 5, 5, 14, 1, 5, 7, 5, 3, 5, 9, 5, 9, 5, 5,..."
...,...,...,...,...,...,...
47954,Sentence: 9995,"[Opposition, leader, Mir, Hossein, Mousavi, ha...","[NNP, NN, NNP, NNP, NNP, VBZ, VBN, PRP, VBZ, T...","[O, O, O, B-per, I-per, O, O, O, O, O, O, O, O...","[30340, 25217, 28150, 16639, 11793, 28397, 256...","[5, 5, 5, 14, 1, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,..."
47955,Sentence: 9996,"[On, Thursday, ,, Iranian, state, media, publi...","[IN, NNP, ,, JJ, NN, NNS, VBN, DT, NN, IN, DT,...","[O, B-tim, O, B-gpe, O, O, O, O, O, O, O, O, B...","[8153, 9982, 2891, 15322, 13761, 28853, 28832,...","[5, 7, 5, 9, 5, 5, 5, 5, 5, 5, 5, 5, 6, 2, 5, ..."
47956,Sentence: 9997,"[Following, Iran, 's, disputed, June, 12, elec...","[VBG, NNP, POS, JJ, NNP, CD, NNS, ,, NNS, NNS,...","[O, B-geo, O, O, B-tim, I-tim, O, O, O, O, O, ...","[18735, 15816, 12347, 964, 15428, 30225, 31955...","[5, 3, 5, 5, 7, 16, 5, 5, 5, 5, 5, 5, 5, 5, 5,..."
47957,Sentence: 9998,"[Since, then, ,, authorities, have, held, publ...","[IN, RB, ,, NNS, VBP, VBN, JJ, NNS, IN, DT, VB...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[23424, 17710, 2891, 2572, 1271, 27404, 21480,...","[5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, ..."


In [17]:
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

def get_pad_train_test_val(data_group, data):

    n_token = len(list(set(data['Word'].to_list())))
    n_tag = len(list(set(data['Tag'].to_list())))

    tokens = data_group['Word_idx'].to_list()
    maxlen = max([len(s) for s in tokens])
    pad_tokens = pad_sequences(tokens, maxlen=maxlen, dtype='int32', padding='post', value=n_token -1)

    tags = data_group['Tag_idx'].to_list()
    pad_tags = pad_sequences(tags, maxlen=maxlen, dtype='int32', padding='post', value=tag2idx['O'])
    n_tags = len(tag2idx)
    pad_tags = [to_categorical(i, num_classes=n_tags) for i in pad_tags]

    tokens_, test_tokens, tags_, test_tags = train_test_split(pad_tokens, pad_tags, test_size=0.1, train_size=0.9, random_state=2020)
    train_tokens, val_tokens, train_tags, val_tags = train_test_split(tokens_,tags_,test_size = 0.25,train_size =0.75, random_state=2020)

    print(
    'train_tokens length:', len(train_tokens),
    '\ntrain_tags:', len(train_tags),
    '\ntest_tokens length:', len(test_tokens),
    '\ntest_tags:', len(test_tags),
    '\nval_tokens:', len(val_tokens),
    '\nval_tags:', len(val_tags),
)
    return train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags

In [18]:
import numpy as np
import tensorflow
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from tensorflow.keras.utils import plot_model
np.random.seed(1)
tensorflow.random.set_seed(2)

In [19]:
train_tokens, val_tokens, test_tokens, train_tags, val_tags, test_tags = get_pad_train_test_val(data_group, data)

train_tokens length: 32372 
train_tags: 32372 
test_tokens length: 4796 
test_tags: 4796 
val_tokens: 10791 
val_tags: 10791


In [20]:
input_dim = len(list(set(data['Word'].to_list())))+1
output_dim  = 64
input_length = max([len(s) for s in data_group['Word_idx'].to_list()])
n_tags = len(tag2idx)

In [21]:
def get_bilstml_lstml_model():
    model = Sequential()
    #Embedding layer
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))
    #Bidirectional LSTM
    model.add(Bidirectional(LSTM(units=output_dim, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), merge_mode='concat'))
    #LSTM layer
    model.add(LSTM(units=output_dim, return_sequences=True, dropout=0.5, recurrent_dropout=0.5))
    #timeDistributed layer
    model.add(TimeDistributed(Dense(n_tags, activation='relu')))
    #Optimizer
    adam = tensorflow.keras.optimizers.Adam(lr=0.0005, beta_1=0.2, beta_2=0.999)

    #Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()

    return model

In [24]:
from keras.callbacks import EarlyStopping

def train_model(X_train, y_train, X_val, y_val, model):

    # Đặt up EarlyStopping callback
    early_stopping = EarlyStopping(monitor='val_loss', # giám sát mất mát của tập validation
                                min_delta=0.001,    # sự thay đổi tối thiểu được coi là cải thiện
                                patience=5,         # số lượng epochs không cải thiện sau đó dừng
                                verbose=1,          # in thông báo khi dừng
                                mode='min',         # dừng khi giá trị được giám sát không giảm
                                restore_best_weights=True) # khôi phục trọng số từ epoch tốt nhất
    loss = list()
    for i in range(25):
        #fit model for one epoch on this sequence
        hist = model.fit(X_train, y_train, batch_size=1000, verbose = 1, epochs=2, validation_data=(X_val, y_val), callbacks=[early_stopping])
        loss.append(hist.history['loss'][0])
        print(f'"Done {i} iteration(s)"')
    return loss

In [25]:
results = pd.DataFrame()
model_bilstm_lstml = get_bilstml_lstml_model()
plot_model(model_bilstm_lstml)
results['with_add_lstm'] = train_model(train_tokens, np.array(train_tags), val_tokens, np.array(val_tags), model_bilstm_lstml)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 104, 64)           2251456   
                                                                 
 bidirectional_1 (Bidirecti  (None, 104, 128)          66048     
 onal)                                                           
                                                                 
 lstm_3 (LSTM)               (None, 104, 64)           49408     
                                                                 
 time_distributed_1 (TimeDi  (None, 104, 17)           1105      
 stributed)                                                      
                                                                 
Total params: 2368017 (9.03 MB)
Trainable params: 2368017 (9.03 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/2
33/33 

In [28]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
text = nlp('I am Dai. I am a student')
displacy.render(text, style = 'ent', jupyter=True)